# **FIFA Data Analysis**

During this data analysis, I had a few objectives to complete.

The instructions were to create your a football club using the following guidelines:
* My club doesn't have a team yet.
* The team will need to hire players for the roster.
* I must select the players using a data based approach.
* I have to create a report to recommend 14-16 players for the roster.
* Assume a budget for hiring players to be limmited.
* The report has to help management/stakeholders in decision making.

**I started by importing the pandas library.**

In [2]:
import pandas as pd

**Then, I added in my dataset.**

In [3]:
df = pd.read_csv('fifa_eda_stats.csv')

**Data Cleaning**

I first removed the Loaned From column and players with no club. I assumed these to be irrelevant with little effect to the crucial data needed to form the best club within a reasonable budget.

In [4]:
# Remove Loaned From column since it only applies to a few records
df.drop('Loaned From', axis=1, inplace=True)

# Remove players with no club
df.dropna(subset=['Club'], inplace=True)

I then cleaned the Value, Wage, and Release Clause columns to remove the euro symbol as well as any existing "K" and "M" in the cells indicating thousands or millions.

Then, I converted the value type to a float.

In [5]:
# Clean € cells by removing special characters
df['Value'] = df['Value'].str.replace('€', '')
df['Wage'] = df['Wage'].str.replace('€', '')
df['Release Clause'] = df['Release Clause'].str.replace('€', '')

# Convert values to numeric
df['Value'] = df['Value'].str.replace('K', 'e3').str.replace('M', 'e6').astype(float)
df['Wage'] = df['Wage'].str.replace('K', 'e3').str.replace('M', 'e6').astype(float)
df['Release Clause'] = df['Release Clause'].str.replace('K', 'e3').str.replace('M', 'e6').astype(float)

Continuing this, I dropped any players that had nothing in the Value column, as these entries would be crucial to forming the team.

I changed the height from feet and inches into inches. Though this was not necessary for my analysis, I decided to standardize the formatting of all cells. Similarly, I removed the "lbs" from all cells in the Weight column.

In [6]:
# Drop players with no value
df = df[df['Value'] != 0]

# Height to inches
height_split = df['Height'].str.split("'", expand=True)
feet = height_split[0].astype(float)
inches = height_split[1].astype(float)
df['Height'] = feet * 12 + inches
df.dropna(subset=['Height'], inplace=True)  # Drop rows with missing height values

# Fix weight to remove lbs
df['Weight'] = df['Weight'].str.replace('lbs', '').astype(int)

**Player Selection**

I used a few filters that I deemed necessary for my player selection.

First, given that the objective is to hire a brand new roster for a new club, it makes sense to look for players under 30, as they are closer to their prime and would potentially have a longer future in the sport.

Also, I removed duplicate entries.

In [7]:
# Filter players by age under 30
df = df[df['Age'] < 30]

# Remove duplicate entries based on ID
df.drop_duplicates(subset='ID', keep='first', inplace=True)

Then, I categorized and filtered the existing values for players' position by using broader categories than the specific positions allow.

In [8]:
# Filter players
goalkeepers = df[df['Position'].isin(['GK'])]
center_backs = df[df['Position'].isin(['CB', 'LCB', 'RCB'])]
full_backs = df[df['Position'].isin(['LB', 'RB', 'LWB', 'RWB'])]
central_midfielders = df[df['Position'].isin(['CM', 'CDM'])]
wing_midfielders = df[df['Position'].isin(['LM', 'RM', 'LCM', 'RCM', 'LDM', 'RDM'])]
attacking_midfielders = df[df['Position'] == 'CAM']
strikers = df[df['Position'].isin(['ST', 'LS', 'RS'])]
wingers = df[df['Position'].isin(['LW', 'RW', 'LF', 'RF'])]

I then decided on attributes that were most indicative of performance in each specific position, and sorted those values.

In [9]:
# Rank players
goalkeepers_ranked = goalkeepers.sort_values(by=['GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'Strength', 'Jumping', 'Height', 'Reactions', 'ShortPassing', 'LongPassing'], ascending=False)
center_backs_ranked = center_backs.sort_values(by=['HeadingAccuracy', 'Interceptions', 'Marking', 'StandingTackle', 'SlidingTackle', 'Strength', 'Jumping'], ascending=False)
full_backs_ranked = full_backs.sort_values(by=['Crossing', 'Dribbling', 'Acceleration', 'SprintSpeed', 'Stamina', 'Interceptions', 'Marking', 'StandingTackle'], ascending=False)
central_midfielders_ranked = central_midfielders.sort_values(by=['ShortPassing', 'LongPassing', 'Vision', 'BallControl', 'Stamina', 'Interceptions', 'Marking'], ascending=False)
wing_midfielders_ranked = wing_midfielders.sort_values(by=['Dribbling', 'Crossing', 'Acceleration', 'SprintSpeed', 'Stamina', 'BallControl'], ascending=False)
attacking_midfielders_ranked = attacking_midfielders.sort_values(by=['Vision', 'LongPassing', 'Dribbling', 'ShotPower', 'LongShots', 'Positioning', 'BallControl'], ascending=False)
strikers_ranked = strikers.sort_values(by=['Finishing', 'HeadingAccuracy', 'Positioning', 'Acceleration', 'SprintSpeed', 'ShotPower', 'LongShots'], ascending=False)
wingers_ranked = wingers.sort_values(by=['Dribbling', 'Crossing', 'Acceleration', 'SprintSpeed', 'Agility', 'BallControl', 'Finishing'], ascending=False)

I created a list to store my selected players then defined how many players in each position I wanted to for my selection.

In [10]:
# Select players
selected_players = []
total_value = 0

# Define desired number of players for each position
desired_positions = {
    'GK': 1,
    'CB': 2,
    'LB': 1,
    'RB': 1,
    'CM': 2,
    'CDM': 1,
    'LM': 1,
    'RM': 1,
    'CAM': 1,
    'ST': 1,
    'LW': 1,
    'RW': 1
}

I then used a for loop to select my players based on their Overall and the desired positions. I wanted to keep in mind the budget and used a limit of 6 millions euros.

In [11]:
# Select top players based on overall rating and position
for position, count in desired_positions.items():
    if position == 'GK':
        position_players = goalkeepers_ranked
    elif position == 'CB':
        position_players = center_backs_ranked
    elif position == 'LB' or position == 'RB':
        position_players = full_backs_ranked
    elif position == 'CM' or position == 'CDM':
        position_players = central_midfielders_ranked
    elif position == 'LM' or position == 'RM':
        position_players = wing_midfielders_ranked
    elif position == 'CAM':
        position_players = attacking_midfielders_ranked
    elif position == 'ST':
        position_players = strikers_ranked
    elif position == 'LW' or position == 'RW':
        position_players = wingers_ranked
    else:
        position_players = pd.DataFrame()

    for _, player in position_players.iterrows():
        if total_value + player['Value'] <= 600000000.0 and count > 0 and player['ID'] not in selected_players:
            selected_players.append(player['ID'])
            total_value += player['Value']
            count -= 1

        if len(selected_players) >= 16 or count == 0:
            break

Then, I filtered my data frame to only include the selected players, calculated the total value of the roster, and printed my new team.

In [12]:
selected_players_df = df[df['ID'].isin(selected_players)]

# Calculate total budget
total_budget = selected_players_df['Value'].sum()
print("Total Budget: €", total_budget)

# Reset index and display selected players with the specified columns
selected_players_df.reset_index(drop=True, inplace=True)
columns_to_print = ['ID', 'Name', 'Age', 'Club', 'Value', 'Overall']
selected_players_df.index += 1  # Start index from 1
print(selected_players_df[columns_to_print])

Total Budget: € 598100000.0
        ID           Name  Age                     Club       Value  Overall
1   193080         De Gea   27        Manchester United  72000000.0       91
2   183277      E. Hazard   27                  Chelsea  93000000.0       91
3   202126        H. Kane   24        Tottenham Hotspur  83500000.0       89
4   190460     C. Eriksen   26        Tottenham Hotspur  73500000.0       88
5   190483  Douglas Costa   27                 Juventus  46500000.0       86
6   189509         Thiago   27        FC Bayern München  45500000.0       86
7   180206      M. Pjanić   28                 Juventus  44000000.0       86
8   184267     Y. Brahimi   28                 FC Porto  39000000.0       85
9   205498       Jorginho   26                  Chelsea  38000000.0       84
10  201956        S. Sané   27            FC Schalke 04  18500000.0       82
11  186345    K. Trippier   27        Tottenham Hotspur  18500000.0       82
12  212187         P. Max   24              FC A